## <center>**Sincronização de Banco de Dados** - Grupo 2<center>


Imports

In [67]:
import pandas as pd
import psycopg2
import os
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

True

Conexão com o Banco Origem 

In [68]:
conn_source = psycopg2.connect(
    host=os.getenv("SOURCE_HOST"),
    database="db Copa ",
    user=os.getenv("SOURCE_USER"),
    password=os.getenv("SOURCE_PASSWORD"),
    port=os.getenv("SOURCE_PORT")
)

try:
    cur_source = conn_source.cursor()
    print("Conexão estabelecida com sucesso!")
except Exception as e:
    print("Erro ao conectar ao banco de dados:\n", e)

Conexão estabelecida com sucesso!


Conexão com o Banco Target

In [69]:
conn_target = psycopg2.connect(
    host=os.getenv("TARGET_HOST"),
    database=os.getenv("TARGET_DATABASE"),
    user=os.getenv("TARGET_USER"),
    password=os.getenv("TARGET_PASSWORD"),
    port=os.getenv("TARGET_PORT")
)

try:
    cur_target = conn_target.cursor()
    print("Conexão estabelecida com sucesso!")
except Exception as e:
    print("Erro ao conectar ao banco de dados:\n", e)

Conexão estabelecida com sucesso!


Funções de INSERT

In [70]:
def insert_paises(df, cur_target, conn_target):
    query = """
        INSERT INTO paises (
            id, nome, continente, qnt_titulos,
            criado_em, atualizado_em
        )
        VALUES (%s, %s, %s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP)
        ON CONFLICT (id) DO NOTHING;
    """
    data = [
        (
            int(ii["id"]),
            ii["nome"],
            ii["continente"],
            int(ii["qnt_titulos"]),
        )
        for i, ii in df.iterrows()
    ]

    cur_target.executemany(query, data)
    conn_target.commit()

In [71]:
def insert_jogadores(df, cur_target, conn_target):
    query = """
        INSERT INTO jogadores (
            id, nome, data_nascimento, posicao, altura, peso, pais_id,
            criado_em, atualizado_em
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP)
        ON CONFLICT (id) DO NOTHING;
    """
    data = [
        (
            int(ii["id"]),
            ii["nome"],
            ii["data_nascimento"],
            ii["posicao"],
            float(ii["altura"]),
            float(ii["peso"]),
            int(ii["pais_id"]),
        )
        for i, ii in df.iterrows()
    ]

    cur_target.executemany(query, data)
    conn_target.commit()


In [72]:
def insert_partidas(df, cur_target, conn_target):
    query = """
        INSERT INTO partidas (
            id, data_partida, estadio, time_casa_id, time_visitante_id,
            placar_casa, placar_visitante, qnt_faltas_casa, qnt_faltas_visitante,
            criado_em, atualizado_em
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP)
        ON CONFLICT (id) DO NOTHING;
    """
    data = [
        (
            int(ii["id"]),
            ii["data_partida"],
            ii["estadio"],
            int(ii["time_casa_id"]),
            int(ii["time_visitante_id"]),
            int(ii["placar_casa"]),
            int(ii["placar_visitante"]),
            int(ii["qnt_faltas_casa"]),
            int(ii["qnt_faltas_visitante"]),
        )
        for i, ii in df.iterrows()
    ]

    cur_target.executemany(query, data)
    conn_target.commit()

In [73]:
def insert_campeonatos(df, cur_target, conn_target):
    query = """
        INSERT INTO campeonatos (
            id, ano, sede, campeao_id, vice_id,
            criado_em, atualizado_em
        )
        VALUES (%s, %s, %s, %s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP)
        ON CONFLICT (id) DO NOTHING;
    """
    data = [
        (
            int(ii["id"]),
            int(ii["ano"]),
            ii["sede"],
            ii["campeao_id"],
            ii["vice_id"],
        )
        for i, ii in df.iterrows()
    ]

    cur_target.executemany(query, data)
    conn_target.commit()


In [74]:
def insert_fases(df, cur_target, conn_target):
    query = """
        INSERT INTO fases (
            id, nome,
            criado_em, atualizado_em
        )
        VALUES (%s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP)
        ON CONFLICT (id) DO NOTHING;
    """
    data = [
        (
            int(ii["id"]),
            ii["nome"],
        )
        for i, ii in df.iterrows()
    ]

    cur_target.executemany(query, data)
    conn_target.commit()

In [75]:
def insert_chaveamentos(df, cur_target, conn_target):
    query = """
        INSERT INTO chaveamento (
            id, campeonato_id, fase_id, partida_id, vencedor_id,
            criado_em, atualizado_em
        )
        VALUES (%s, %s, %s, %s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP)
        ON CONFLICT (id) DO NOTHING;
    """
    data = [
        (
            int(ii["id"]),
            int(ii["campeonato_id"]),
            int(ii["fase_id"]),
            int(ii["partida_id"]),
            ii["vencedor_id"],
        )
        for i, ii in df.iterrows()
    ]

    cur_target.executemany(query, data)
    conn_target.commit()

Funções de UPDATE

In [76]:
def update_paises(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = """
            UPDATE paises
            SET nome = %s,
                continente = %s,
                qnt_titulos = %s,
                atualizado_em = CURRENT_TIMESTAMP
            WHERE id = %s;
        """
        values = (
            ii["nome"],
            ii["continente"],
            ii["qnt_titulos"],
            ii["id"]
        )
        cur_target.execute(query, values)

    conn_target.commit()

In [77]:
def update_jogadores(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = """
            UPDATE jogadores
            SET nome = %s,
                data_nascimento = %s,
                posicao = %s,
                altura = %s,
                peso = %s,
                pais_id = %s,
                atualizado_em = CURRENT_TIMESTAMP
            WHERE id = %s;
        """
        values = (
            ii["nome"],
            ii["data_nascimento"],
            ii["posicao"],
            ii["altura"],
            ii["peso"],
            ii["pais_id"],
            ii["id"]
        )
        cur_target.execute(query, values)

    conn_target.commit()

In [78]:
def update_partidas(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = """
            UPDATE partidas
            SET data_partida = %s,
                estadio = %s,
                time_casa_id = %s,
                time_visitante_id = %s,
                placar_casa = %s,
                placar_visitante = %s,
                qnt_faltas_casa = %s,
                qnt_faltas_visitante = %s,
                atualizado_em = CURRENT_TIMESTAMP
            WHERE id = %s;
        """
        values = (
            ii["data_partida"],
            ii["estadio"],
            ii["time_casa_id"],
            ii["time_visitante_id"],
            ii["placar_casa"],
            ii["placar_visitante"],
            ii["qnt_faltas_casa"],
            ii["qnt_faltas_visitante"],
            ii["id"]
        )
        cur_target.execute(query, values)

    conn_target.commit()

In [79]:
def update_campeonatos(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = """
            UPDATE campeonatos
            SET ano = %s,
                sede = %s,
                campeao_id = %s,
                vice_id = %s,
                atualizado_em = CURRENT_TIMESTAMP
            WHERE id = %s;
        """
        values = (
            ii["ano"],
            ii["sede"],
            ii["campeao_id"],
            ii["vice_id"],
            ii["id"]
        )
        cur_target.execute(query, values)

    conn_target.commit()

In [80]:
def update_fases(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = """
            UPDATE fases
            SET nome = %s,
                atualizado_em = CURRENT_TIMESTAMP
            WHERE id = %s;
        """
        values = (
            ii["nome"],
            ii["id"]
        )
        cur_target.execute(query, values)

    conn_target.commit()

In [81]:
def update_chaveamentos(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = """
            UPDATE chaveamento
            SET campeonato_id = %s,
                fase_id = %s,
                partida_id = %s,
                vencedor_id = %s,
                atualizado_em = CURRENT_TIMESTAMP
            WHERE id = %s;
        """
        values = (
            ii["campeonato_id"],
            ii["fase_id"],
            ii["partida_id"],
            ii["vencedor_id"],
            ii["id"]
        )
        cur_target.execute(query, values)

    conn_target.commit()

In [82]:
def delete_paises(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = "DELETE FROM paises WHERE id = %s"
        cur_target.execute(query, (ii['id'],))
        conn_target.commit()


def delete_jogadores(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = "DELETE FROM jogadores WHERE id = %s"
        cur_target.execute(query, (ii['id'],))
        conn_target.commit()
    conn_target.commit()



def delete_partidas(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = "DELETE FROM partidas WHERE id = %s"
        cur_target.execute(query, (ii['id'],))
        conn_target.commit()


def delete_campeonatos(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = "DELETE FROM campeonatos WHERE id = %s"
        cur_target.execute(query, (ii['id'],))
        conn_target.commit()

def delete_fases(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = "DELETE FROM fases WHERE id = %s"
        cur_target.execute(query, (ii['id'],))
        conn_target.commit()



def delete_chaveamentos(df, cur_target, conn_target):
    for i, ii in df.iterrows():
        query = "DELETE FROM chaveamento WHERE id = %s"
        cur_target.execute(query, (ii['id'],))
        conn_target.commit()

In [83]:
funcs = {
    "paises": {
        "insert": insert_paises,
        "update": update_paises,
        "delete": delete_paises
    },
    "jogadores": {
        "insert": insert_jogadores,
        "update": update_jogadores,
        "delete": delete_jogadores
    },
    "partidas": {
        "insert": insert_partidas,
        "update": update_partidas,
        "delete": delete_partidas
    },
    "campeonatos": {
        "insert": insert_campeonatos,
        "update": update_campeonatos,
        "delete": delete_campeonatos
    },
    "fases": {
        "insert": insert_fases,
        "update": update_fases,
        "delete": delete_fases
    },
    "chaveamento": {
        "insert": insert_chaveamentos,
        "update": update_chaveamentos,
        "delete": delete_chaveamentos
    }
}

Funções de Controle da Última Sincronização

In [84]:
def get_last_sinc(conn_target):
    last_sinc = pd.read_sql_query("SELECT MAX(ultima_sincronizacao) FROM controle_sincronizacao;", conn_target)
    return last_sinc.iloc[0, 0] or datetime(1900,1,1)

def set_last_sinc(cur_target, conn_target):
    data = datetime.now()
    cur_target.execute("INSERT INTO controle_sincronizacao (ultima_sincronizacao) VALUES (%s)", (data,))
    conn_target.commit()

Funções para localizar as inserções, atualizações e remoções de registros

In [85]:
def get_inserts(conn_source, table, last_sinc):
    query = f"SELECT * FROM {table} WHERE criado_em > '{last_sinc}'"
    return pd.read_sql(query, conn_source)

def get_updates(conn_source, table, last_sinc):
    query = f"""
        SELECT * FROM {table}
        WHERE atualizado_em > '{last_sinc}'
        AND criado_em <= '{last_sinc}'
    """
    return pd.read_sql(query, conn_source)

def get_deletes(conn_source, table, last_sinc):
    query = f"""
        SELECT * FROM {table}
        WHERE data_exclusao > '{last_sinc}'
    """
    return pd.read_sql(query, conn_source)

Função de Sincronização de uma tabela

In [86]:
def sinc_table(conn_source, conn_target, cur_target, table, funcs, last_sinc):
    inserts = get_inserts(conn_source, table, last_sinc)
    updates = get_updates(conn_source, table, last_sinc)
    deletes = get_deletes(conn_source, table, last_sinc)

    if not inserts.empty:
        funcs[table]["insert"](inserts, cur_target, conn_target)
        print(f"{table}: {len(inserts)} inseridos")

    if not updates.empty:
        funcs[table]["update"](updates, cur_target, conn_target)
        print(f"{table}: {len(updates)} atualizados")

    if not deletes.empty:
        funcs[table]["delete"](deletes, cur_target, conn_target)
        print(f"{table}: {len(deletes)} deletados")


Função com Loop para Sincronizar todas as tabelas

In [87]:
def sinc_all_tables():
    last_sinc = get_last_sinc(conn_target)

    for table in funcs.keys():
        sinc_table(conn_source, conn_target, cur_target, table, funcs, last_sinc)

    set_last_sinc(cur_target, conn_target)

In [88]:
# Executa a sincronização
sinc_all_tables()

C:\Users\rafaelcruz-ieg\AppData\Local\Temp\ipykernel_12596\1966082394.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  last_sinc = pd.read_sql_query("SELECT MAX(ultima_sincronizacao) FROM controle_sincronizacao;", conn_target)
C:\Users\rafaelcruz-ieg\AppData\Local\Temp\ipykernel_12596\2807154314.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn_source)
C:\Users\rafaelcruz-ieg\AppData\Local\Temp\ipykernel_12596\2807154314.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_

fases: 1 inseridos


C:\Users\rafaelcruz-ieg\AppData\Local\Temp\ipykernel_12596\2807154314.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn_source)
C:\Users\rafaelcruz-ieg\AppData\Local\Temp\ipykernel_12596\2807154314.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn_source)
C:\Users\rafaelcruz-ieg\AppData\Local\Temp\ipykernel_12596\2807154314.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn_source)
